In [25]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean, desc
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()

### Import des CSVs

In [27]:
df_top_streamers = spark.read.option("header", True).csv("/home/elisabeth/Documents/hadoop/twitchdata-update.csv")
df_GB_target = spark.read.option("header", True).csv("/home/elisabeth/Documents/hadoop/ENGB_target.csv")
df_GB_edges = spark.read.option("header", True).csv("/home/elisabeth/Documents/hadoop/ENGB_edges.csv")
df_edges = spark.read.option("header", True).csv("/home/elisabeth/Documents/hadoop/GephiData/07.02.2021.02.40.42EDGELIST.csv")

### Edges streamers abonnés

In [ ]:
df_edges.sort("Source").limit(5).toPandas()

### Streamers les plus suivis

In [ ]:
df_edges.groupBy("Target").count().sort(desc("count")).show()

In [ ]:
#df_top_streamers.printSchema()
print("top streamers - count: ", df_top_streamers.count())
df_top_streamers.show()

### Moyenne des followers des top streamers

In [ ]:
# cast string en int df_top_streamers + check autres types de colonnes

In [ ]:
print(df_top_streamers.select(mean("Followers"))
# pas utiliser collect -> machine locale

### Streamers par ordre décroissant de followers

In [ ]:
df_top_streamers.sort(desc("Peak viewers")).show()

### Vieux trucs

### Streamers par ordre décroissant de followers

In [34]:
df_top_streamers.sort(desc("Peak viewers")).show()

+-----------------+-------------------+--------------------+------------+---------------+---------+----------------+------------+---------+------+----------+
|          Channel|Watch time(Minutes)|Stream time(minutes)|Peak viewers|Average viewers|Followers|Followers gained|Views gained|Partnered|Mature|  Language|
+-----------------+-------------------+--------------------+------------+---------------+---------+----------------+------------+---------+------+----------+
|        ESL_DOTA2|          661049190|              212010|       99858|           4714|   337177|           91323|    14784068|     True| False|   English|
|피닉스박1 (xkwhd)|          205225605|              179595|        9938|           1139|   142209|           50675|     4794915|     True| False|    Korean|
|  龜狗 (gueigotv)|          194262165|              134895|        9918|           1373|   165232|           25466|     6005834|     True|  True|   Chinese|
|DreamHackDota2_RU|          152461155|                613

### Vieux trucs

In [5]:
print("UK target - count: ", df_GB_target.count())
df_GB_target.show()

UK target - count:  7126
+---------+----+------+------+-------+------+
|       id|days|mature| views|partner|new_id|
+---------+----+------+------+-------+------+
| 73045350|1459| False|  9528|  False|  2299|
| 61573865|1629|  True|  3615|  False|   153|
|171688860| 411|  True| 46546|  False|   397|
|117338954| 953|  True|  5863|  False|  5623|
|135804217| 741|  True|  5594|  False|  5875|
| 50273956|1820| False|  4594|  False|  3346|
| 40189534|2070| False| 18460|  False|  6930|
|126853728| 846|  True|   204|  False|  6367|
|118869338| 936|  True|  1064|  False|  1434|
|115599547| 973| False|313084|  False|  4470|
| 94973589|1198| False|   791|  False|  5122|
|113241013| 993|  True|   364|  False|  4790|
|119663818| 931|  True|  3994|  False|  4919|
| 81085785|1354| False|  3956|  False|   575|
| 55592461|1720| False|  2833|  False|   639|
| 64700169|1573|  True| 16054|  False|  7012|
| 36174401|2219| False|  3950|  False|  6445|
| 38079222|2142|  True| 89458|   True|  1170|
| 1323325

In [6]:
print("UK edges - count: ", df_GB_target.count())
df_GB_edges.show()

UK edges - count:  7126
+----+----+
|from|  to|
+----+----+
|6194| 255|
|6194| 980|
|6194|2992|
|6194|2507|
|6194| 986|
|6194|4003|
|   0|  82|
|  15| 343|
|  15|4282|
|  15|5442|
|  15|1162|
|  15|3401|
|7106|6211|
|7106|6611|
|7106|1348|
|7106|2239|
|   2|5842|
|   3|3363|
|   3|1277|
|   3|3739|
+----+----+
only showing top 20 rows

